## Centre of mass and COM-centred coordinates

We have a molecule with $N$ atoms.

- Atom $i$ has mass $m_i$ and Cartesian position $\mathbf{R}_i \in \mathbb{R}^3$.
- The total mass is
$$
M_{\text{tot}} = \sum_{i=1}^{N} m_i.
$$

### Centre of mass (COM)
The position of the centre of mass is
$$
\mathbf{R}_{\text{COM}} = \frac{1}{M_{\text{tot}}} \sum_{i=1}^{N} m_i \, \mathbf{R}_i.
$$

In [2]:
from __future__ import annotations
from typing import Tuple
import numpy as np

In [3]:
def center_of_mass(R: np.ndarray, masses: np.ndarray) -> np.ndarray:
    """
    Compute the centre of mass (COM).

    Args:
        R: (N,3) Cartesian coordinates [Å].
        masses: (N,) atomic masses [amu or any consistent unit].

    Returns:
        com: (3,) COM coordinates [Å].
    """
    R = np.asarray(R, dtype=float)
    masses = np.asarray(masses, dtype=float)

    if R.ndim != 2 or R.shape[1] != 3:
        raise ValueError("R must have shape (N,3)")
    if masses.shape != (R.shape[0],):
        raise ValueError("masses must have shape (N,)")

    Mtot = np.sum(masses)
    if Mtot == 0.0:
        raise ValueError("Total mass is zero, cannot compute COM")

    com = (masses[:, None] * R).sum(axis=0) / Mtot
    return com


### COM-centred coordinates
We define coordinates relative to the COM:
$$
\mathbf{r}_i = \mathbf{R}_i - \mathbf{R}_{\text{COM}}.
$$

Stacking these for all atoms gives a new geometry with the COM at the origin:
$$
\{\mathbf{r}_1, \mathbf{r}_2, \ldots, \mathbf{r}_N\}.
$$

`shift_to_com()` returns $(\mathbf{r}_i)$ and also the original $\mathbf{R}_{\text{COM}}$.


In [4]:
def shift_to_com(R: np.ndarray, masses: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """
    Return coordinates with COM at the origin.

    Args:
        R: (N,3) Cartesian coordinates [Å].
        masses: (N,) masses.

    Returns:
        Rc: (N,3) COM-centred coordinates, i.e. R - COM.
        com: (3,) original COM.
    """
    com = center_of_mass(R, masses)
    Rc = np.asarray(R, dtype=float) - com[None, :]
    return Rc, com

## Testing

In [8]:
import numpy as np
from ase.build import molecule

target = 'H2O'
atoms = molecule(target)
symbols = atoms.get_chemical_symbols()
R = atoms.get_positions() 

masses = atoms.get_masses() 

com = center_of_mass(R, masses)
Rc, com2 = shift_to_com(R, masses)

print("Name: ",target)
print("Original COM:\n", com)
print("Recomputed COM from Rc:\n", center_of_mass(Rc, masses)) # This second COM should be [0,0,0]

Name:  H2O
Original COM:
 [0.       0.       0.052531]
Recomputed COM from Rc:
 [0.00000000e+00 0.00000000e+00 1.23255401e-17]


## Build rigid basis vectors (translations + rotations around COM)

In [9]:
def rigid_translation_vectors(N: int) -> np.ndarray:
    T = np.zeros((3*N, 3), float)
    for a in range(3):
        for i in range(N):
            T[3*i + a, a] = 1.0
    return T

In [10]:
def rigid_rotation_vectors(Rc: np.ndarray) -> np.ndarray:
    N = Rc.shape[0]
    Rot = np.zeros((3*N, 3), float)
    axes = np.eye(3)
    for a in range(3):
        for i in range(N):
            Rot[3*i:3*i+3, a] = np.cross(axes[a], Rc[i])
    return Rot